# 基于 Docker Compose 搭建 Neo4j

本 Notebook 将演示如何使用 Docker Compose 快速部署 Neo4j 图数据库，并使用 Python 驱动进行连接测试。

## 前置条件
- 已安装 Docker
- 已安装 Docker Compose (Docker Desktop 通常自带)
- Python 环境 (如果需要运行测试代码)

## 1. 编写 docker-compose.yml 文件

我们使用 Jupyter 的 `%%writefile` 魔术命令将配置直接写入当前目录下的 `docker-compose.yml` 文件中。

**配置说明**：
- `NEO4J_AUTH`: 设置初始账号密码 (默认用户 neo4j，密码设置为 password)。
- `volumes`: 将数据持久化挂载到当前目录的 `data` 文件夹。

In [ ]:
%%writefile docker-compose.yml
version: '3.8'

services:
  neo4j:
    image: neo4j:5.15.0-community
    container_name: neo4j_container
    ports:
      - "7474:7474" # HTTP Browser
      - "7687:7687" # Bolt Port
    environment:
      - NEO4J_AUTH=neo4j/password
      - NEO4J_PLUGINS=["apoc"]  # 可选：安装 APOC 插件
    volumes:
      - ./data:/data
      - ./logs:/logs
    restart: unless-stopped

## 2. 启动服务

使用 shell 命令启动容器。`-d` 表示后台运行。

In [ ]:
!docker compose up -d

## 3. 检查运行状态

查看容器是否成功启动。状态应为 `Up`。

In [ ]:
!docker ps --filter "name=neo4j_container"

## 4. 等待服务就绪
Neo4j 启动需要几秒钟时间。我们可以简单地等待一会儿，或者查看日志。

In [ ]:
import time
print("Waiting for Neo4j to start...")
time.sleep(15) # 等待15秒确保Bolt端口已准备好
print("Ready!")

## 5. Python 连接测试 (可选)

使用官方驱动 `neo4j` 测试数据库是否可用。
如果未安装驱动，请先运行 `!pip install neo4j`。

In [ ]:
# 安装驱动 (如果已安装可跳过)
!pip install neo4j

In [ ]:
from neo4j import GraphDatabase

# 配置连接信息
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "password")

def test_connection():
    try:
        with GraphDatabase.driver(URI, auth=AUTH) as driver:
            driver.verify_connectivity()
            print("✅ 连接成功！Neo4j 正在运行。")
            
            # 简单创建一个节点测试
            records, summary, keys = driver.execute_query(
                "MERGE (p:Person {name: $name}) RETURN p.name AS name",
                name="Alice",
                database_="neo4j",
            )
            for record in records:
                print(f"已确认节点存在: {record['name']}")
                
    except Exception as e:
        print(f"❌ 连接失败: {e}")

test_connection()

## 6. 访问 Web 界面

你现在可以通过浏览器访问 Neo4j Browser：

- **地址**: [http://localhost:7474](http://localhost:7474)
- **用户名**: neo4j
- **密码**: password

## 7. 停止与清理

实验结束后，可以停止并删除容器。

In [ ]:
# 停止并移除容器
!docker compose down